In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
from pathlib import Path
from multiprocessing.dummy import Pool as ThreadPool
from collections import defaultdict
from natsort import natsorted
import tsfresh as tf
import sklearn

In [3]:
pd.set_option('max_columns', None)

### Data loading

In [4]:
import os
import glob

dataset_path = 'data_feats_90_c/'
csv_files = glob.glob(dataset_path+'*.csv')

all_df = []

for filename in csv_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    all_df.append(df)

df = pd.concat(all_df, axis=0, ignore_index=True)
df.head(1)

,Unnamed: 0,P-PDG__mean,P-PDG__variance,P-PDG__skewness,P-PDG__kurtosis,"P-PDG__fft_aggregated__aggtype_""centroid""","P-PDG__fft_aggregated__aggtype_""variance""","P-PDG__fft_aggregated__aggtype_""skew""","P-PDG__fft_aggregated__aggtype_""kurtosis""",P-PDG__maximum,P-PDG__minimum,P-PDG__median,P-PDG__quantile__q_0.1,P-PDG__quantile__q_0.2,P-PDG__quantile__q_0.3,P-PDG__quantile__q_0.4,P-PDG__quantile__q_0.6,P-PDG__quantile__q_0.7,P-PDG__quantile__q_0.8,P-PDG__quantile__q_0.9,P-PDG__variation_coefficient,P-PDG__mean_change,P-PDG__mean_second_derivative_central,P-PDG__friedrich_coefficients__coeff_1__m_3__r_30,P-PDG__friedrich_coefficients__coeff_3__m_3__r_30,P-TPT__mean,P-TPT__variance,P-TPT__skewness,P-TPT__kurtosis,"P-TPT__fft_aggregated__aggtype_""centroid""","P-TPT__fft_aggregated__aggtype_""variance""","P-TPT__fft_aggregated__aggtype_""skew""","P-TPT__fft_aggregated__aggtype_""kurtosis""",P-TPT__maximum,P-TPT__minimum,P-TPT__median,P-TPT__quantile__q_0.1,P-TPT__quantile__q_0.2,P-TPT__quantile__q_0.3,P-TPT__quantile__q_0.4,P-TPT__quantile__q_0.6,P-TPT__quantile__q_0.7,P-TPT__quantile__q_0.8,P-TPT__quantile__q_0.9,P-TPT__variation_coefficient,P-TPT__mean_change,P-TPT__mean_second_derivative_central,P-TPT__friedrich_coefficients__coeff_1__m_3__r_30,P-TPT__friedrich_coefficients__coeff_3__m_3__r_30,T-TPT__mean,T-TPT__variance,T-TPT__skewness,T-TPT__kurtosis,"T-TPT__fft_aggregated__aggtype_""centroid""","T-TPT__fft_aggregated__aggtype_""variance""","T-TPT__fft_aggregated__aggtype_""skew""","T-TPT__fft_aggregated__aggtype_""kurtosis""",T-TPT__maximum,T-TPT__minimum,T-TPT__median,T-TPT__quantile__q_0.1,T-TPT__quantile__q_0.2,T-TPT__quantile__q_0.3,T-TPT__quantile__q_0.4,T-TPT__quantile__q_0.6,T-TPT__quantile__q_0.7,T-TPT__quantile__q_0.8,T-TPT__quantile__q_0.9,T-TPT__variation_coefficient,T-TPT__mean_change,T-TPT__mean_second_derivative_central,T-TPT__friedrich_coefficients__coeff_1__m_3__r_30,T-TPT__friedrich_coefficients__coeff_3__m_3__r_30,P-MON-CKP__mean,P-MON-CKP__variance,P-MON-CKP__skewness,P-MON-CKP__kurtosis,"P-MON-CKP__fft_aggregated__aggtype_""centroid""","P-MON-CKP__fft_aggregated__aggtype_""variance""","P-MON-CKP__fft_aggregated__aggtype_""skew""","P-MON-CKP__fft_aggregated__aggtype_""kurtosis""",P-MON-CKP__maximum,P-MON-CKP__minimum,P-MON-CKP__median,P-MON-CKP__quantile__q_0.1,P-MON-CKP__quantile__q_0.2,P-MON-CKP__quantile__q_0.3,P-MON-CKP__quantile__q_0.4,P-MON-CKP__quantile__q_0.6,P-MON-CKP__quantile__q_0.7,P-MON-CKP__quantile__q_0.8,P-MON-CKP__quantile__q_0.9,P-MON-CKP__variation_coefficient,P-MON-CKP__mean_change,P-MON-CKP__mean_second_derivative_central,P-MON-CKP__friedrich_coefficients__coeff_1__m_3__r_30,P-MON-CKP__friedrich_coefficients__coeff_3__m_3__r_30,T-JUS-CKP__mean,T-JUS-CKP__variance,T-JUS-CKP__skewness,T-JUS-CKP__kurtosis,"T-JUS-CKP__fft_aggregated__aggtype_""centroid""","T-JUS-CKP__fft_aggregated__aggtype_""variance""","T-JUS-CKP__fft_aggregated__aggtype_""skew""","T-JUS-CKP__fft_aggregated__aggtype_""kurtosis""",T-JUS-CKP__maximum,T-JUS-CKP__minimum,T-JUS-CKP__median,T-JUS-CKP__quantile__q_0.1,T-JUS-CKP__quantile__q_0.2,T-JUS-CKP__quantile__q_0.3,T-JUS-CKP__quantile__q_0.4,T-JUS-CKP__quantile__q_0.6,T-JUS-CKP__quantile__q_0.7,T-JUS-CKP__quantile__q_0.8,T-JUS-CKP__quantile__q_0.9,T-JUS-CKP__variation_coefficient,T-JUS-CKP__mean_change,T-JUS-CKP__mean_second_derivative_central,T-JUS-CKP__friedrich_coefficients__coeff_1__m_3__r_30,T-JUS-CKP__friedrich_coefficients__coeff_3__m_3__r_30,QGL__mean,QGL__variance,QGL__skewness,QGL__kurtosis,"QGL__fft_aggregated__aggtype_""centroid""","QGL__fft_aggregated__aggtype_""variance""","QGL__fft_aggregated__aggtype_""skew""","QGL__fft_aggregated__aggtype_""kurtosis""",QGL__maximum,QGL__minimum,QGL__median,QGL__quantile__q_0.1,QGL__quantile__q_0.2,QGL__quantile__q_0.3,QGL__quantile__q_0.4,QGL__quantile__q_0.6,QGL__quantile__q_0.7,QGL__quantile__q_0.8,QGL__quantile__q_0.9,QGL__variation_coefficient,QGL__mean_change,QGL__mean_second_derivative_central,QGL__friedrich_c

In [5]:
df.drop(['Unnamed: 0'], axis='columns', inplace=True)      #leftover index column, unnecessary

df = df[df.class_code != 7]                                #paper does not use event class 7
df['class_code'].replace({8: 7}, inplace=True)


### Handling NA values

In [6]:
def print_na_sum(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        nans = df.isna().sum(axis=0)
        print(nans[nans!=0])

print_na_sum(df)

P-PDG__fft_aggregated__aggtype_"centroid"                 8571
P-PDG__fft_aggregated__aggtype_"variance"                 8571
P-PDG__fft_aggregated__aggtype_"skew"                    37244
P-PDG__fft_aggregated__aggtype_"kurtosis"                37244
P-PDG__variation_coefficient                              8571
P-PDG__friedrich_coefficients__coeff_1__m_3__r_30        20675
P-PDG__friedrich_coefficients__coeff_3__m_3__r_30        20675
P-TPT__fft_aggregated__aggtype_"centroid"                   64
P-TPT__fft_aggregated__aggtype_"variance"                   64
P-TPT__fft_aggregated__aggtype_"skew"                    30450
P-TPT__fft_aggregated__aggtype_"kurtosis"                30450
P-TPT__variation_coefficient                                64
P-TPT__friedrich_coefficients__coeff_1__m_3__r_30        12623
P-TPT__friedrich_coefficients__coeff_3__m_3__r_30        12623
T-TPT__fft_aggregated__aggtype_"centroid"                 6514
T-TPT__fft_aggregated__aggtype_"variance"              


Removing features with too many NA values

In [7]:
df.drop(['QGL__variation_coefficient'], axis='columns', inplace=True)
df.drop(list(df.filter(regex = 'friedrich')), axis = 1, inplace = True)
df.drop(list(df.filter(regex = 'fft_aggregated')), axis = 1, inplace = True)

In [8]:
print_na_sum(df)

P-PDG__variation_coefficient        8571
P-TPT__variation_coefficient          64
T-TPT__variation_coefficient        6514
P-MON-CKP__variation_coefficient    1200
T-JUS-CKP__variation_coefficient    1819
dtype: int64


Imputing leftover na values, for each class separately

In [9]:
imputed_df_list = []
for i in df['class_code'].unique():
    query = 'class_code == ' + str(i)
    imputed = tf.utilities.dataframe_functions.impute(df.query(query))
    imputed_df_list.append(imputed)
imputed_df = pd.concat(imputed_df_list)

/home/marko/.local/lib/python3.8/site-packages/pandas/core/generic.py:9032: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(cond, other, inplace, axis, level, errors=errors)


In [10]:
print_na_sum(imputed_df)

Series([], dtype: int64)


In [11]:
imputed_df = imputed_df.sample(frac=1).reset_index(drop=True)
sample_df = imputed_df#.sample(frac=0.01, random_state=42)
X = sample_df.iloc[:,:-1].to_numpy()
y = sample_df.iloc[:,-1].to_numpy()
sample_df.shape

(51764, 108)

In [12]:
X

array([[ 2.30907290e+07,  1.67859110e+08, -9.87866905e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.27047155e+07,  3.45259423e+07,  1.46226592e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 2.66925701e+07,  1.93413580e+01, -3.38947192e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.11729161e+07,  1.56645921e+04,  1.66925410e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.88900044e+07,  8.32314277e+10, -1.84615348e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [13]:
y

array([1, 1, 0, ..., 5, 2, 5])

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [15]:
from sklearn.model_selection import KFold   

#todo: include window size into grid search

window_size_hp = [300, 600, 900]
regularization_hp = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]

In [16]:
from sklearn.model_selection import KFold   
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

kf = KFold(n_splits=5)

### Logistic regression, SVM

In [21]:
from sklearn.linear_model import SGDClassifier

for clf in ['log', 'hinge']:
    if clf == 'log': 
        print('Logistic regression')
    if clf == 'hinge':
        print('Support vector machine')
    for r in regularization_hp:
        i = 0
        for train_index, test_index in kf.split(X_train):
            X_k_train, X_k_test = X_train[train_index], X_train[test_index]
            y_k_train, y_k_test = y_train[train_index], y_train[test_index]
            scaler = StandardScaler().fit(X_k_train)
            X_k_train = scaler.transform(X_k_train)
            X_k_test = scaler.transform(X_k_test)
            log_reg = SGDClassifier(loss='log', max_iter=10000, alpha=r).fit(X_k_train, y_k_train)
            h = log_reg.predict(X_k_test)
            X_test_scaled = scaler.transform(X_test)
            h_test = log_reg.predict(X_test_scaled)
            print('r='+str(r)+', i-th fold='+str(i)+', k-fold f1 score='+str(f1_score(y_k_test, h, average='weighted'))+
                 ' , test f1 score=' + str(f1_score(y_test, h_test, average='weighted')))
            i += 1

Logistic regression
r=1e-07, i-th fold=0, k-fold f1 score=0.8401872542682329 , test f1 score=0.8379039457595059
r=1e-07, i-th fold=1, k-fold f1 score=0.8626394353931612 , test f1 score=0.8541881572912307
r=1e-07, i-th fold=2, k-fold f1 score=0.8602098051803506 , test f1 score=0.8616852415193939
r=1e-07, i-th fold=3, k-fold f1 score=0.8690558940956953 , test f1 score=0.8621495813121065
r=1e-07, i-th fold=4, k-fold f1 score=0.8562633167860713 , test f1 score=0.8564897760975014
r=1e-06, i-th fold=0, k-fold f1 score=0.817977266226078 , test f1 score=0.8217819525605519
r=1e-06, i-th fold=1, k-fold f1 score=0.7794296301820831 , test f1 score=0.7792715181736267
r=1e-06, i-th fold=2, k-fold f1 score=0.8700028655628602 , test f1 score=0.869948342789524
r=1e-06, i-th fold=3, k-fold f1 score=0.8606078664433205 , test f1 score=0.8483905605123464
r=1e-06, i-th fold=4, k-fold f1 score=0.8748608562379685 , test f1 score=0.8732824219474736
r=1e-05, i-th fold=0, k-fold f1 score=0.8706413658551747 , tes

### Decision trees

In [26]:
from sklearn.tree import DecisionTreeClassifier

complexity_hp = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
for ccp in complexity_hp:
    i = 0
    for train_index, test_index in kf.split(X_train):
        X_k_train, X_k_test = X_train[train_index], X_train[test_index]
        y_k_train, y_k_test = y_train[train_index], y_train[test_index]
        scaler = StandardScaler().fit(X_k_train)
        X_k_train = scaler.transform(X_k_train)
        X_k_test = scaler.transform(X_k_test)
        tree = DecisionTreeClassifier(random_state=42, ccp_alpha=ccp).fit(X_k_train, y_k_train)
        h = tree.predict(X_k_test)
        X_test_scaled = scaler.transform(X_test)
        h_test = log_reg.predict(X_test_scaled)
        print('r='+str(r)+', i-th fold='+str(i)+', k-fold f1 score='+str(f1_score(y_k_test, h, average='weighted'))+
            ' , test f1 score=' + str(f1_score(y_test, h_test, average='weighted')))
        i += 1

r=0.1, i-th fold=0, k-fold f1 score=0.9940536320209209 , test f1 score=0.8068014709418998
r=0.1, i-th fold=1, k-fold f1 score=0.9941825484201936 , test f1 score=0.8048281940610577
r=0.1, i-th fold=2, k-fold f1 score=0.9947488991979021 , test f1 score=0.8060421049179922
r=0.1, i-th fold=3, k-fold f1 score=0.9946105391912002 , test f1 score=0.8060359728995646
r=0.1, i-th fold=4, k-fold f1 score=0.9938823443060995 , test f1 score=0.8044059048081371
r=0.1, i-th fold=0, k-fold f1 score=0.9926522899178287 , test f1 score=0.8068014709418998
r=0.1, i-th fold=1, k-fold f1 score=0.9937758827232576 , test f1 score=0.8048281940610577
r=0.1, i-th fold=2, k-fold f1 score=0.994465656303886 , test f1 score=0.8060421049179922
r=0.1, i-th fold=3, k-fold f1 score=0.9938902172053287 , test f1 score=0.8060359728995646
r=0.1, i-th fold=4, k-fold f1 score=0.9920852648635484 , test f1 score=0.8044059048081371
r=0.1, i-th fold=0, k-fold f1 score=0.981009837844463 , test f1 score=0.8068014709418998
r=0.1, i-th 

### Random forest

In [37]:
from sklearn.ensemble import RandomForestClassifier

n_trees_hp = [50, 100, 150, 175]
max_depth_hp = [5, 7, 10, None]
n_feats_at_splits_hp = [5, 10, 15]   #pitati za komentar o ovom hiperparametru

for n_trees in n_trees_hp:
    for max_depth in max_depth_hp:
        for n_feats_at_splits in n_feats_at_splits_hp:
            i = 0
            for train_index, test_index in kf.split(X_train):
                X_k_train, X_k_test = X_train[train_index], X_train[test_index]
                y_k_train, y_k_test = y_train[train_index], y_train[test_index]
                scaler = StandardScaler().fit(X_k_train)
                X_k_train = scaler.transforAdaBoostClassifierin)
                X_k_test = scaler.transform(X_k_test)
                trees = RandomForestClassifier(random_state=42, n_estimators=n_trees,
                                              max_depth=max_depth, min_samples_split=n_feats_at_splits)
                trees.fit(X_k_train, y_k_train)
                h = trees.predict(X_k_test)
                X_test_scaled = scaler.transform(X_test)
                h_test = trees.predict(X_test_scaled)
                print('n_feats_at_split=' + str(n_feats_at_splits) +', n_trees=' + str(n_trees) + 
                      ', depth=' + str(max_depth) + ', i-th fold='+str(i)+', k-fold f1 score='+
                      str(f1_score(y_k_test, h, average='weighted'))+
                    ' , test f1 score=',str(f1_score(y_test, h_test, average='weighted')))
                i += 1

n_feats_at_split=5, n_trees=50, depth=5, i-th fold=0, k-fold f1 score=0.9428118881520631 , test f1 score= 0.9440303074698622
n_feats_at_split=5, n_trees=50, depth=5, i-th fold=1, k-fold f1 score=0.9476230942921152 , test f1 score= 0.9424597276917482
n_feats_at_split=5, n_trees=50, depth=5, i-th fold=2, k-fold f1 score=0.9596419577723524 , test f1 score= 0.9525051646155107
n_feats_at_split=5, n_trees=50, depth=5, i-th fold=3, k-fold f1 score=0.9474223498231915 , test f1 score= 0.9429092491046382
n_feats_at_split=5, n_trees=50, depth=5, i-th fold=4, k-fold f1 score=0.9444142671855857 , test f1 score= 0.9392993067259389
n_feats_at_split=10, n_trees=50, depth=5, i-th fold=0, k-fold f1 score=0.9414505764068896 , test f1 score= 0.9428974856112711
n_feats_at_split=10, n_trees=50, depth=5, i-th fold=1, k-fold f1 score=0.9473017843035493 , test f1 score= 0.942205373587905
n_feats_at_split=10, n_trees=50, depth=5, i-th fold=2, k-fold f1 score=0.9532609787438677 , test f1 score= 0.947486637607197

KeyboardInterrupt: 

### Linear Discriminant Analysis, Quadratic Discriminant Analysis

In [38]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
    
print('LDA')
i = 0
for train_index, test_index in kf.split(X_train):
    X_k_train, X_k_test = X_train[train_index], X_train[test_index]
    y_k_train, y_k_test = y_train[train_index], y_train[test_index]
    scaler = StandardScaler().fit(X_k_train)
    X_k_train = scaler.transform(X_k_train)
    X_k_test = scaler.transform(X_k_test)
    clf = LinearDiscriminantAnalysis().fit(X_k_train, y_k_train)
    h = log_reg.predict(X_k_test)
    X_test_scaled = scaler.transform(X_test)
    h_test = log_reg.predict(X_test_scaled)
    print('i-th fold='+str(i)+', k-fold f1 score='+str(f1_score(y_k_test, h, average='weighted'))+
         ' , test f1 score=' + str(f1_score(y_test, h_test, average='weighted')))
    i += 1
print('\nQDA')
i = 0
for train_index, test_index in kf.split(X_train):
    X_k_train, X_k_test = X_train[train_index], X_train[test_index]
    y_k_train, y_k_test = y_train[train_index], y_train[test_index]
    scaler = StandardScaler().fit(X_k_train)
    X_k_train = scaler.transform(X_k_train)
    X_k_test = scaler.transform(X_k_test)
    clf = QuadraticDiscriminantAnalysis().fit(X_k_train, y_k_train)
    h = log_reg.predict(X_k_test)
    X_test_scaled = scaler.transform(X_test)
    h_test = log_reg.predict(X_test_scaled)
    print('i-th fold='+str(i)+', k-fold f1 score='+str(f1_score(y_k_test, h, average='weighted'))+
         ' , test f1 score=' + str(f1_score(y_test, h_test, average='weighted')))
    i += 1

LDA
i-th fold=0, k-fold f1 score=0.8049100769337282 , test f1 score=0.8068014709418998
i-th fold=1, k-fold f1 score=0.8116570256046773 , test f1 score=0.8048281940610577
i-th fold=2, k-fold f1 score=0.803570102497663 , test f1 score=0.8060421049179922
i-th fold=3, k-fold f1 score=0.8122185723865437 , test f1 score=0.8060359728995646
i-th fold=4, k-fold f1 score=0.808061793552293 , test f1 score=0.8044059048081371

QDA


/home/marko/miniconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:873: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


i-th fold=0, k-fold f1 score=0.8049100769337282 , test f1 score=0.8068014709418998


/home/marko/miniconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:873: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


i-th fold=1, k-fold f1 score=0.8116570256046773 , test f1 score=0.8048281940610577


...

i-th fold=2, k-fold f1 score=0.803570102497663 , test f1 score=0.8060421049179922


/home/marko/miniconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:873: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


i-th fold=3, k-fold f1 score=0.8122185723865437 , test f1 score=0.8060359728995646


/home/marko/miniconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:873: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


i-th fold=4, k-fold f1 score=0.808061793552293 , test f1 score=0.8044059048081371


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

n_trees_hp = [100, 250, 400, 550]
max_depth_hp = [5, 3, 1]
lr_hp = [0.01, 0.1, 1]

for n_trees in n_trees_hp:
    for max_depth in max_depth_hp:
        for lr in lr_hp:
            i = 0
            for train_index, test_index in kf.split(X_train):
                X_k_train, X_k_test = X_train[train_index], X_train[test_index]
                y_k_train, y_k_test = y_train[train_index], y_train[test_index]
                scaler = StandardScaler().fit(X_k_train)
                X_k_train = scaler.transform(X_k_train)
                X_k_test = scaler.transform(X_k_test)
                trees = AdaBoostClassifier(random_state=42, n_estimators=n_trees,
                                            base_estimator=DecisionTreeClassifier(max_depth=max_depth))
                trees.fit(X_k_train, y_k_train)
                h = trees.predict(X_k_test)
                X_test_scaled = scaler.transform(X_test)
                h_test = trees.predict(X_test_scaled)
                print('n_trees=' + str(n_trees) +', max_depth=' + str(max_depth) + 
                      ', lr=' + str(lr) + ', i-th fold='+str(i)+', k-fold f1 score='+
                      str(f1_score(y_k_test, h, average='weighted'))+
                    ' , test f1 score=',str(f1_score(y_test, h_test, average='weighted')))
                i += 1

n_trees=100, max_depth=1, lr=0.01, i-th fold=0, k-fold f1 score=0.43006572979711116 , test f1 score= 0.43552210419404913

n_trees=100, max_depth=1, lr=0.01, i-th fold=1, k-fold f1 score=0.43796987623221034 , test f1 score= 0.43628167707258503

n_trees=100, max_depth=1, lr=0.01, i-th fold=2, k-fold f1 score=0.43264596110154757 , test f1 score= 0.4367323580643448

n_trees=100, max_depth=1, lr=0.01, i-th fold=3, k-fold f1 score=0.49794200553633955 , test f1 score= 0.48359977830756967

n_trees=100, max_depth=1, lr=0.01, i-th fold=4, k-fold f1 score=0.489498734181628 , test f1 score= 0.4832915753085723

n_trees=100, max_depth=1, lr=0.1, i-th fold=0, k-fold f1 score=0.43006572979711116 , test f1 score= 
0.43552210419404913

n_trees=100, max_depth=1, lr=0.1, i-th fold=1, k-fold f1 score=0.43796987623221034 , test f1 score= 0.43628167707258503

n_trees=100, max_depth=1, lr=0.1, i-th fold=2, k-fold f1 score=0.43264596110154757 , test f1 score= 0.4367323580643448

n_trees=100, max_depth=1, lr=0.1, i-th fold=3, k-fold f1 score=0.49794200553633955 , test f1 score= 0.48359977830756967

n_trees=100, max_depth=1, lr=0.1, i-th fold=4, k-fold f1 score=0.489498734181628 , test f1 score= 0.4832915753085723

n_trees=100, max_depth=1, lr=1, i-th fold=0, k-fold f1 score=0.43006572979711116 , test f1 score= 0.43552210419404913